# Pairwise Matching su blockingW2V

In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial.distance import euclidean
from itertools import combinations

C:\Users\Riccardo De Cesaris\AppData\Local\Temp\ipykernel_20484\3249245688.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Calcolo della Distanza "Intra-Cluster" Media

In [2]:
def compute_mean_distance(elements):
    pairs = list(combinations(elements,2))
    # Calcola la distanza euclidea tra ciascuna coppia di vettori
    distances = [euclidean(pair[0], pair[1]) for pair in pairs]

    # Calcola la media delle distanze
    mean_distance = np.mean(distances)

    #print("Media distanza tra i vettori di embedding:", mean_distance)

    return mean_distance

## Caricamento dello schema mediato con cluster (Strategia Embedding Word2Vec)

In [3]:
schema_mediato_con_blocks = pd.read_csv("./../datasets/MediatedSchemaConBlocks_W2V.csv")

In [4]:
num_blocks = schema_mediato_con_blocks['block'].nunique()
print(num_blocks)

2167


In [5]:
bert_vectors = pd.read_csv("./../datasets/MediatedSchemaVectors.csv")

## AgglomerativeClustering su Vettori estratti dai Record con BERT

In [6]:
stats = {}

In [7]:
for i in range(0,num_blocks):
    records_cluster = schema_mediato_con_blocks[schema_mediato_con_blocks['block']==i]
    if(len(records_cluster)>1):
        vectors_cluster = bert_vectors.loc[records_cluster.index].copy() #lista degli indici di riga per il cluster corrente
        distance_threshold = compute_mean_distance(vectors_cluster.values.tolist())
        model = AgglomerativeClustering(n_clusters=None,distance_threshold=0.33*distance_threshold).fit(vectors_cluster)
        stats[i] = {}
        vectors_cluster['cluster'] = model.labels_
        for j in range(0,max(model.labels_)):
            stats[i][j] = vectors_cluster[vectors_cluster['cluster']==j].index.tolist()
    else:
        stats[i] = {}
        stats[i][0] = records_cluster.index.tolist()

In [9]:
elementi_con_dimensione_maggiore_di_1 = sum(len(d) > 1 for d in stats.values())
elementi_con_dimensione_maggiore_di_1

2039

## Salvataggio su File

In [10]:
with open("./results_clustersW2V.txt", "w") as file:
    # Iterare sul dizionario e scrivere ogni coppia chiave-valore su una nuova riga
    for chiave, valore in stats.items():
        file.write(f"{chiave}: {valore}\n")